# **Hyperparameters tuning and Evaluation**

In [19]:
#Imports
import numpy as np
import pandas as pd
import random
import math
import plots
from matplotlib import pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import SVDpp
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SlopeOne
from surprise import accuracy
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from sklearn.model_selection import KFold as skFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from Vince_helpers import *

my_seed = 200
random.seed(my_seed)
np.random.seed(my_seed)

In [5]:
#Load data train file
train = pd.read_csv('data_train.csv')
train = to_surprise(train)

In [6]:
#Split data into models training part and blender training part
models = train.sample(frac=0.8, random_state=200)
blend = train.drop(models.index)

In [7]:
#Load both datasets into surprise as datasets and trainsets objects
reader = Reader(rating_scale=(1, 5))
models_surp = Dataset.load_from_df(models, reader)
models_surp_train = models_surp.build_full_trainset()
blend_surp = Dataset.load_from_df(blend, reader)
blend_surp_train = blend_surp.build_full_trainset()
#Load blend train set as a testset for models performance evaluation
blend_surp_test = blend_surp_train.build_testset()

We grid search the best hyperparameters for each models individually on the models training set. We evaluate each combination based on a K=3 Fold CV procedure (folds are set to be the same every time for reproducibility). Then we will pick the combinations yielding the smallest average RMSE over the folds and refit the models on the whole models training data. Note that some models do not require tuning (global mean, user/item mean, slopeone) and will therefore be fitted directly on the whole models train dataset.

Algorithm tuning:
- Establish Grid
- Run Grid search
- Extract best hyperparameters combination based on average RMSE
- Retrain on the whole models train set

In [8]:
#Basic solutions: global mean, user mean and movie mean
#Here we compute the mean or dataframes of means per users or movies
mean = global_mean(models)
users = user_mean(models)
movies = movie_mean(models)

In [69]:
#Baseline
grid_baseline = {'bsl_options': {'method': ['sgd'],
                              'reg': [10**-i for i in range(8,9)]},
                 'verbose':[False]
                }
gs_baseline = GridSearchCV(BaselineOnly, grid_baseline, measures=['rmse'], 
                           cv=KFold(n_splits=3, random_state=200, shuffle=False))
gs_baseline.fit(models_surp)
print('Best Hyperparameters: ', gs_baseline.best_params['rmse'])
algo_baseline = gs_baseline.best_estimator['rmse']
algo_baseline.fit(models_surp_train)

Best Hyperparameters:  {'bsl_options': {'method': 'sgd', 'reg': 1e-08}, 'verbose': False}


In [40]:
#SVD with baseline
grid_SVDb = {'reg_all': [10**-i for i in range(1,2)], 'biased':[True], 'n_factors':[100], 'random_state':[200]}
gs_SVDb = GridSearchCV(SVD, grid_SVDb, measures=['rmse'], 
                       cv=KFold(n_splits=3, random_state=200, shuffle=False))
gs_SVDb.fit(models_surp)
print('Best Hyperparameters: ', gs_SVDb.best_params['rmse'])
algo_SVDb = gs_SVDb.best_estimator['rmse']
algo_SVDb.fit(models_surp_train)

Best Hyperparameters:  {'reg_all': 0.01, 'biased': True, 'n_factors': 100, 'random_state': 200}


In [ ]:
#SVD without baseline
grid_SVD = {'reg_all':[10**-i for i in range(1,2)], 'biased':[False], 'n_factors':[100], 'random_state':[200]}
gs_SVD = GridSearchCV(SVD, grid_SVD, measures=['rmse'], 
                      cv=KFold(n_splits=3, random_state=200, shuffle=False))
gs_SVD.fit(models_surp)
print('Best Hyperparameters: ', gs_SVD.best_params['rmse'])
algo_SVD = gs_SVD.best_estimator['rmse']
algo_SVD.fit(models_surp_train)

In [ ]:
#SVD++
grid_SVDpp = {'reg_all':[0.01], 'n_factors':[20], 'random_state':[200]}
gs_SVDpp = GridSearchCV(SVDpp, grid_SVDpp, measures=['rmse'], 
                        cv=KFold(n_splits=3, random_state=200, shuffle=False))
gs_SVDpp.fit(models_surp)
print('Best Hyperparameters: ', gs_SVDpp.best_params['rmse'])
algo_SVDpp = gs_SVDpp.best_estimator['rmse']
algo_SVDpp.fit(models_surp_train)

In [10]:
#Slope One
algo_slope_one = SlopeOne()
algo_slope_one.fit(models_surp_train)

In [70]:
#KNN user
grid_knn_user = {'bsl_options': {'method': ['sgd'],
                              'reg': [10**-i for i in range(1,2)]},
                              'k': [40],
                              'sim_options': {'name': ['pearson_baseline'],
                              'min_support': [1],
                              'user_based': [True]}
                }
gs_knn_user = GridSearchCV(KNNBaseline, grid_knn_user, measures=['rmse'], 
                        cv=KFold(n_splits=3, random_state=200, shuffle=False))
gs_knn_user.fit(models_surp)
print('Best Hyperparameters: ', gs_knn_user.best_params['rmse'])
algo_knn_user = gs_knn_user.best_estimator['rmse']
algo_knn_user.fit(models_surp_train)

In [46]:
#KNN movie
grid_knn_movie = {'bsl_options': {'method': ['sgd'],
                              'reg': [10**-i for i in range(3,4)]},
                              'k': [60],
                              'sim_options': {'name': ['pearson_baseline'],
                              'min_support': [1],
                              'user_based': [False]}
                }
gs_knn_movie = GridSearchCV(KNNBaseline, grid_knn_movie, measures=['rmse'], 
                        cv=KFold(n_splits=3, random_state=200, shuffle=False))
gs_knn_movie.fit(models_surp)
print('Best Hyperparameters: ', gs_knn_movie.best_params['rmse'])
algo_knn_movie = gs_knn_movie.best_estimator['rmse']
algo_knn_movie.fit(models_surp_train)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Best Hyperparameters:  {'bsl_options': {'method': 'sgd', 'reg': 0.001}, 'k': 60, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Now that every algorithm has been fitted on the whole models train dataset we will evaluate their performance (RMSE) on the blend train dataset. This set is therefore also used as a validation set for individual models.

In [45]:
#Baseline
predictions_baseline = algo_baseline.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_baseline, verbose=False))

RMSE on validation set:  1.0032986221959859


In [41]:
#SVD with baseline
predictions_SVDb = algo_SVDb.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_SVDb, verbose=False))

RMSE on validation set:  1.0647191794539996


In [ ]:
#SVD without baseline
predictions_SVD = algo_SVD.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_SVD, verbose=False))

In [ ]:
#SVD++
predictions_SVDpp = algo_SVDpp.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_SVDpp, verbose=False))

In [ ]:
#Slope One
predictions_slope_one = algo_slope_one.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_slope_one, verbose=False))

In [ ]:
#KNN user
predictions_knn_user = algo_knn_user.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_knn_user, verbose=False))

In [47]:
#KNN movie
predictions_knn_movie = algo_knn_movie.test(blend_surp_test)
print('RMSE on validation set: ', accuracy.rmse(predictions_knn_movie, verbose=False))

RMSE on validation set:  0.9911173306703703


In [ ]:
#Recover ids and estimations for each algorithm
uids = [pred.uid for pred in predictions_baseline]
mids = [pred.iid for pred in predictions_baseline]
ruis = [pred.r_ui for pred in predictions_baseline]
est_baseline = [pred.est for pred in predictions_baseline]
est_SVDb = [pred.est for pred in predictions_SVDb]
est_SVD = [pred.est for pred in predictions_SVD]
#est_SVDpp = [pred.est for pred in predictions_SVDpp]
est_slope_one = [pred.est for pred in predictions_slope_one]
#est_knn_user = [pred.est for pred in predictions_knn_user]
est_knn_movie = [pred.est for pred in predictions_knn_movie]
est_global = [mean for i in range(len(ruis))]
est_user_mean = [predict_user(u, users, mean) for u in uids]
est_movie_mean = [predict_movie(m, movies, mean) for m in mids]

In [12]:
#RMSE on validation set for basic solutions
global_rmse = math.sqrt(sum([(a-b)**2 for (a,b) in zip(ruis, est_global)])/len(ruis))
user_rmse = math.sqrt(sum([(a-b)**2 for (a,b) in zip(ruis, est_user_mean)])/len(ruis))
movie_rmse = math.sqrt(sum([(a-b)**2 for (a,b) in zip(ruis, est_movie_mean)])/len(ruis))
print('Global mean RMSE on validation set: ', global_rmse)
print('User mean RMSE on validation set: ', user_rmse)
print('Movie mean RMSE on validation set: ', movie_rmse)

Global mean RMSE on validation set:  1.119984536592249
User mean RMSE on validation set:  1.095988679007401
Movie mean RMSE on validation set:  1.0300326000124342


We will now use the blend train set to train our model blending algorithm. We model the estimated rating as a linear combination of estimated ratings for each model. We will resort to ridge regression to compute the weights of our model. The best ridge hyperparameter is picked based on a 3 fold CV procedure (objective function = average RMSE) operated on 75% of the blender train set. 

In [50]:
#Build dataframe containing true ratings and estimations for each algorithm
est_baseline = np.array(est_baseline)
est_global = np.array(est_global)
est_user_mean = np.array(est_user_mean)
est_movie_mean = np.array(est_movie_mean)
est_knn_movie = np.array(est_knn_movie)
#est_knn_user = np.array(est_knn_user)
est_slope_one = np.array(est_slope_one)
est_SVDb = np.array(est_SVDb)
est_SVD = np.array(est_SVD)
#est_SVDpp = np.array(est_SVDpp)

'''
X = np.column_stack((est_global, est_user_mean, est_movie_mean, est_baseline, 
                     est_knn_movie, est_knn_user, est_slope_one,
                     est_SVDb, est_SVD, est_SVDpp))
'''

X = np.column_stack((est_global, est_user_mean, est_movie_mean, est_baseline, 
                     est_knn_movie, est_slope_one,
                     est_SVDb, est_SVD))

y = np.array(ruis)

We split the blend train set into a training set and a validation set

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=200)

We perform the grid search on the training set (i.e infer best lambda and weights)

In [52]:
cv_ridge = skFold(n_splits=3, random_state=200)
gs_ridge = RidgeCV(alphas=[10**-i for i in range(-5,10)], fit_intercept=False, scoring="neg_mean_squared_error", cv=cv_ridge)

In [53]:
gs_ridge.fit(X_train, y_train)
print('Best lambda: ', gs_ridge.alpha_)
print('Weights: ', gs_ridge.coef_)

Best lambda:  10.0
Weights:  [-0.04315075 -0.11769155  0.07828077  0.24455342  0.75458397  0.07984848]


We obtain the validation set RMSE for the blending model

In [54]:
preds_blend = gs_ridge.predict(X_test)

In [55]:
blend_rmse = np.sqrt(np.mean((y_test-preds_blend)**2))
print('Model blending RMSE on validation set: ', blend_rmse)

Model blending RMSE on validation set:  0.9855245790114191


We have found the best hyperparameters for each model and the coefficients for the blending algorithm. We will now retrain our algorithms on the whole dataset. Then final predictions will be a linear combination of predictions.